# Assignments Lesson 1

In this notebook we cover the assignments for lesson 1. The second one is particularly instructive.

## First Assignment: 7-card stud

In this assignment we must return the best 5-card hand out of a 7-card hand. This assignment is easy enough, provided one knows `itertools.combinations`.

In [1]:
# CS 212, hw1-1: 7-card stud
#
# -----------------
# User Instructions
#
# Write a function best_hand(hand) that takes a seven
# card hand as input and returns the best possible 5
# card hand. The itertools library has some functions
# that may help you solve this problem.
#
# -----------------
# Grading Notes
# 
# Muliple correct answers will be accepted in cases 
# where the best hand is ambiguous (for example, if 
# you have 4 kings and 3 queens, there are three best
# hands: 4 kings along with any of the three queens).

import itertools

def best_hand(hand):
    "From a 7-card hand, return the best 5 card hand."
    hands = itertools.combinations(hand, 5)
    return max(hands, key=hand_rank)

    
# ------------------
# Provided Functions
# 
# You may want to use some of the functions which
# you have already defined in the unit to write 
# your best_hand function.

def hand_rank(hand):
    "Return a value indicating the ranking of a hand."
    ranks = card_ranks(hand) 
    if straight(ranks) and flush(hand):
        return (8, max(ranks))
    elif kind(4, ranks):
        return (7, kind(4, ranks), kind(1, ranks))
    elif kind(3, ranks) and kind(2, ranks):
        return (6, kind(3, ranks), kind(2, ranks))
    elif flush(hand):
        return (5, ranks)
    elif straight(ranks):
        return (4, max(ranks))
    elif kind(3, ranks):
        return (3, kind(3, ranks), ranks)
    elif two_pair(ranks):
        return (2, two_pair(ranks), ranks)
    elif kind(2, ranks):
        return (1, kind(2, ranks), ranks)
    else:
        return (0, ranks)
    
def card_ranks(hand):
    "Return a list of the ranks, sorted with higher first."
    ranks = ['--23456789TJQKA'.index(r) for r, s in hand]
    ranks.sort(reverse = True)
    return [5, 4, 3, 2, 1] if (ranks == [14, 5, 4, 3, 2]) else ranks

def flush(hand):
    "Return True if all the cards have the same suit."
    suits = [s for r,s in hand]
    return len(set(suits)) == 1

def straight(ranks):
    """Return True if the ordered 
    ranks form a 5-card straight."""
    return (max(ranks)-min(ranks) == 4) and len(set(ranks)) == 5

def kind(n, ranks):
    """Return the first rank that this hand has 
    exactly n-of-a-kind of. Return None if there 
    is no n-of-a-kind in the hand."""
    for r in ranks:
        if ranks.count(r) == n: return r
    return None

def two_pair(ranks):
    """If there are two pair here, return the two 
    ranks of the two pairs, else None."""
    pair = kind(2, ranks)
    lowpair = kind(2, list(reversed(ranks)))
    if pair and lowpair != pair:
        return (pair, lowpair)
    else:
        return None 
    
def test_best_hand():
    assert (sorted(best_hand("6C 7C 8C 9C TC 5C JS".split()))
            == ['6C', '7C', '8C', '9C', 'TC'])
    assert (sorted(best_hand("TD TC TH 7C 7D 8C 8S".split()))
            == ['8C', '8S', 'TC', 'TD', 'TH'])
    assert (sorted(best_hand("JD TC TH 7C 7D 7S 7H".split()))
            == ['7C', '7D', '7H', '7S', 'JD'])
    return 'test_best_hand passes'

print(test_best_hand())


test_best_hand passes


## Assignment 2: Jokers Wild

This assignment is trickier. The instructions are:

> Write a function best_wild_hand(hand) that takes as input a 7-card hand and returns the best 5 card hand. In this problem, it is possible for a hand to include jokers. Jokers will be treated as 'wild cards' which can take any rank or suit of the same color. The  black joker, '?B', can be used as any spade or club and the red joker, '?R', can be used as any heart  or diamond. The itertools library may be helpful. Feel free to  define multiple functions if it helps you solve the problem.

We already have a function that, given a 7-card hand, computes the best 5 card hand. We need a function that can generate all the possible 7-card hands we can get when we replace, say, a black joker with a black card.

Assume we have a hand which can contain zero, one, or two jokers (one red and one black). The idea of Norvig's solution is to generate, for each card, a list of possible replacements. If the card is not a joker, the only replacement is the card itself. If the card is a red joker, any red card is a possible replacement, and similarly for a black joker. Notice that we reason card by card, providing a list of possible replacements for each one of them. In the "best" case, we only get a list of 7 lists, each one containing a card. In the "worst" case, we get a list containing 5 lists of a single card each, and two lists of 13 cards, the red and the black ones.

It makes sense to generate these lists upfront.

In [2]:
allranks = '23456789TJQKA'
red_cards = [r+s for r in allranks for s in 'HD']
black_cards = [r+s for r in allranks for s in 'SC']

The function that replaces a card with the appropriate list takes a card as an input and returns a list as an output.

In [3]:
def replace(card: str) -> list:
    "Replace a card with a list of possible replacements."
    if card == '?R':
        return red_cards
    elif card == '?B':
        return black_cards
    else:
        return [card]

print(replace('TH'))
print(replace('?B'))

['TH']
['2S', '2C', '3S', '3C', '4S', '4C', '5S', '5C', '6S', '6C', '7S', '7C', '8S', '8C', '9S', '9C', 'TS', 'TC', 'JS', 'JC', 'QS', 'QC', 'KS', 'KC', 'AS', 'AC']


Given a hand, we can then generate all the possible hands by using `itertools.product()`. In the simplest case, we have a list of 7 single-card lists. If we process such a list with `itertools.product()`, we obtain a list with the hand, which is a tuple with the 7 cards.

In [4]:
hand1 = "JD TC TH 7C 7D 7S 7H".split()
print(list(map(replace, hand1)))
list(itertools.product(*map(replace, hand1)))

[['JD'], ['TC'], ['TH'], ['7C'], ['7D'], ['7S'], ['7H']]


[('JD', 'TC', 'TH', '7C', '7D', '7S', '7H')]

If the hand contains a jolly, we get a list containing all the possible hands generated by `replace()`.

In [5]:
hand2 = "6C 7C 8C 9C TC 5C ?B".split()
print(list(map(replace, hand2)))
list(itertools.product(*map(replace, hand2)))

[['6C'], ['7C'], ['8C'], ['9C'], ['TC'], ['5C'], ['2S', '2C', '3S', '3C', '4S', '4C', '5S', '5C', '6S', '6C', '7S', '7C', '8S', '8C', '9S', '9C', 'TS', 'TC', 'JS', 'JC', 'QS', 'QC', 'KS', 'KC', 'AS', 'AC']]


[('6C', '7C', '8C', '9C', 'TC', '5C', '2S'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '2C'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '3S'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '3C'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '4S'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '4C'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '5S'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '5C'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '6S'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '6C'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '7S'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '7C'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '8S'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '8C'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '9S'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', '9C'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', 'TS'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', 'JS'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', 'JC'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', 'QS'),
 ('6C', '7C', '8C', '9C', 'TC', '5C', 'QC'),
 ('6C', '7

The function `best_hand()` we wrote above takes a 7-card hand and returns the best 5-card hand contained there.

In [6]:
best_hand(next(itertools.product(*map(replace, hand2))))

('6C', '7C', '8C', '9C', 'TC')

We can then compute `best_hand()` for every card in the list returned by `itertools.product()`.

In [7]:
best_hands = [best_hand(h) for h in itertools.product(*map(replace, hand2))]
print(len(best_hands))
best_hands

26


[('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('7C', '8C', '9C', 'TC', 'JC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC'),
 ('6C', '7C', '8C', '9C', 'TC')]

You can see there are quite a few duplicates. This is why Norvig uses a set.

In [8]:
best_hands = set([best_hand(h) for h in itertools.product(*map(replace, hand2))])
best_hands

{('6C', '7C', '8C', '9C', 'TC'), ('7C', '8C', '9C', 'TC', 'JC')}

Now we only have two hands to select from, and we can pick with the usual `max(..., key)`.

In [9]:
max(best_hands, key=hand_rank)

('7C', '8C', '9C', 'TC', 'JC')

Putting everything together, we can rewrite the `best_wild_hand()` Norvig shows in his video as follows:

In [10]:
def best_wild_hand(hand):
    hands = set(best_hand(h)
                for h in itertools.product(*map(replace, hand)))
    return max(hands, key=hand_rank)

To see if the function works, we run the testing function that comes with the assignment.

In [11]:
def test_best_wild_hand():
    assert (sorted(best_wild_hand("6C 7C 8C 9C TC 5C ?B".split()))
            == ['7C', '8C', '9C', 'JC', 'TC'])
    assert (sorted(best_wild_hand("TD TC 5H 5C 7C ?R ?B".split()))
            == ['7C', 'TC', 'TD', 'TH', 'TS'])
    assert (sorted(best_wild_hand("JD TC TH 7C 7D 7S 7H".split()))
            == ['7C', '7D', '7H', '7S', 'JD'])
    return 'test_best_wild_hand passes'

test_best_wild_hand()

'test_best_wild_hand passes'